In [119]:
!pip install rdkit
!pip install mordred
!pip install pubchempy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [120]:
import pandas as pd
import numpy as np
import pubchempy as pcp
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import rdMolDescriptors
from urllib.request import urlopen
from urllib.parse import quote

Для отработки домашних заданий мы выбрали 6 датасет

In [6]:
df = pd.read_csv('/content/6.csv')

## Data Preparation

Пока что нам нужно, чтобы соединения были в наличии, поэтому удалим все строки, где нет ничего в Name

In [7]:
df = df[~df['Name'].isna()]
df = df.drop_duplicates().reset_index(drop=True)
df

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/S0022-3093(99)00330-0,7/19/1999,Journal of Non-Crystalline Solids,PHOTOINDUCEDCHANGESINLINEARNONLINEAROPTICALPRO...,As20S60Ge20,0.0,NaN,el_cde_tables,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",2.054,n
1,10.1016/j.fct.2006.05.017,6/7/2006,Food and Chemical Toxicology,GENERATIONFORMALDEHYDEINCIGARETTESOVERVIEWRECE...,Propionaldehyde,0.0,NaN,el_cde_tables,CCC=O,3.74,n
2,10.1016/j.mee.2011.01.031,1/18/2011,Microelectronic Engineering,SINUSOIDALPLASMONICCRYSTALSFORBIODETECTIONSENSORS,PEO,0.0,NaN,el_cde_text,"[['O', 1.0], ['PE', 1.0]]",1.47,refractive index
3,10.1016/j.apradiso.2012.04.026,5/2/2012,Applied Radiation and Isotopes,BENCHMARKINGGEANT4FULLSYSTEMSIMULATIONASSOCIAT...,Aluminum,0.0,NaN,el_cde_tables,[AlH2],4.4,Refractive index
4,10.1016/S0924-4247(02)00264-9,9/16/2002,Sensors and Actuators A: Physical,INFLUENCESMATERIALPROPERTIESCERAMICMICROSTEREO...,Alumina,0.0,NaN,el_mylogic,NaN,1.7,Refractive index
...,...,...,...,...,...,...,...,...,...,...,...
4960,10.1016/j.molliq.2011.07.002,7/22/2011,Journal of Molecular Liquids,UVSPECTRALCHANGESFORAZOCOMPOUNDSINPRESENCEDIFF...,iso-Butanol,0.0,NaN,el_cde_tables,NaN,1.3959,n
4961,10.1016/j.asr.2009.06.015,6/26/2009,Advances in Space Research,DIFFERENTAPPROACHESRAYLEIGHOPTICALDEPTHDETERMI...,CO2,0.0,NaN,el_cde_tables,O=C=O,1.000274585,Refractive index
4962,10.1016/S0022-3093(99)00681-X,2/25/2000,Journal of Non-Crystalline Solids,UVTRANSMISSIONRADIATIONINDUCEDDEFECTSINPHOSPHA...,Zn(PO3)2,0.0,NaN,el_cde_tables,"[['O', 6.0], ['P', 2.0], ['Zn', 1.0]]",1.526,Refractive index
4963,10.1016/j.jct.2016.04.016,4/20/2016,The Journal of Chemical Thermodynamics,ASYMMETRICCRITICALITYIONICSOLUTIONCONTAINING1H...,2-propanol,0.0,632.8 nm,el_mylogic,CC(C)O,1.3884,n


С помощью PubChem добавим смайлзы, которых нет в таблице.

In [10]:
def smiles(x):
  try:
    return pcp.get_properties('CanonicalSMILES', x, 'name')[0]['CanonicalSMILES']
  except IndexError:
    return np.nan

df.loc[df['normalised_name'].isna(), 'normalised_name'] = df.loc[df['normalised_name'].isna(),'Name'].apply(smiles)

Заполняется долго, сохраним датасет для дальнешей работы.

In [11]:
df.to_csv('/content/df_1.csv', index=False)

In [14]:
df = pd.read_csv('/content/df_1.csv')

Вещества могут часто повторяться, возьмем уникальные 

In [22]:
df_names = df.loc[df['Name'].drop_duplicates().index, ['Name', 'normalised_name']]

У нас есть несколько проблем:

* некоторых смайлзов нет
* некоторые смайзы неверные и их не распарсят библиотеки
* а некоторые смайлзы неверные, но библиотеки их считают 

С первым мы пока ничего сделать не можем, поработаем с ними дальше.

Со вторыми можно попробовать запустить перевод вещества в смайлз в pubchem и посмотреть такие же ли они, если вещество найдено.

In [43]:
df_names['smiles'] = np.nan
df_names.loc[~df_names['normalised_name'].isna(), 'smiles'] = df_names.loc[~df_names['normalised_name'].isna(),'Name'].apply(smiles)

Несовпадений явно много, какие-то скорее вариант различия смайлзов, какие-то явно дело в ошибке, заменим их

In [31]:
d = df_names[df_names['smiles'] != df_names['normalised_name']]
d[~d['smiles'].isna()]

,Name,normalised_name,smiles
2,PEO,"[['O', 1.0], ['PE', 1.0]]",OO
3,Aluminum,[AlH2],[Al]
5,cyclotriphosphazene,N1PNP=NP1,N1=PN=PN=P1
9,Silicon,[SiH4],[Si]
13,toluene,Cc1ccccc1,CC1=CC=CC=C1
...,...,...,...
1898,ibuprofen,CC(C)Cc1ccc(cc1)C(C)C(O)=O,CC(C)CC1=CC=C(C=C1)C(C)C(=O)O
1900,triptolide,CC(C)[C@]12O[C@H]1[C@@H]3O[C@@]34[C@@]5(C)CCC6...,CC(C)C12C(O1)C3C4(O3)C5(CCC6=C(C5CC7C4(C2O)O7)...
1905,BVD,"[['B', 1.0], ['VD', 1.0]]",C1C(C(OC1N2C=C(C(=O)NC2=O)C=CBr)CO)O
1909,boehmite,[OH-].[O--].[Al+3],O[Al]=O


In [41]:
df_names.loc[d[~d['smiles'].isna()].index, 'normalised_name'] = df_names.loc[d[~d['smiles'].isna()].index, 'smiles']
df_names = df_names.drop('smiles', axis=1)

С третьим вариантом можно запустить rdkit, чтобы он попробовал собрать инстанс молекулы для смайлза

In [44]:
def check_smiles(smile):
    mol = Chem.MolFromSmiles(smile)
    if mol:
      return True
    else:
      return False

df_names['check_smiles'] = np.nan
df_names.loc[~df_names['normalised_name'].isna(), 'check_smiles'] = df_names.loc[~df_names['normalised_name'].isna(),'normalised_name'].apply(check_smiles)

[10:37:59] SMILES Parse Error: syntax error while parsing: [['As',
[10:37:59] SMILES Parse Error: Failed parsing SMILES '[['As',' for input: '[['As','
[10:37:59] SMILES Parse Error: syntax error while parsing: [['Al',
[10:37:59] SMILES Parse Error: Failed parsing SMILES '[['Al',' for input: '[['Al','
[10:37:59] SMILES Parse Error: syntax error while parsing: [['K',
[10:37:59] SMILES Parse Error: Failed parsing SMILES '[['K',' for input: '[['K','
[10:37:59] SMILES Parse Error: syntax error while parsing: [['C',
[10:37:59] SMILES Parse Error: Failed parsing SMILES '[['C',' for input: '[['C','
[10:37:59] SMILES Parse Error: syntax error while parsing: [['Er',
[10:37:59] SMILES Parse Error: Failed parsing SMILES '[['Er',' for input: '[['Er','
[10:37:59] SMILES Parse Error: syntax error while parsing: [['Li',
[10:37:59] SMILES Parse Error: Failed parsing SMILES '[['Li',' for input: '[['Li','
[10:37:59] SMILES Parse Error: syntax error while parsing: [['As',
[10:37:59] SMILES Parse Error: Fa

Некоторые можно заполнить по поиску из `cactus.nci.nih.gov/`

In [47]:
def CIRconvert(x):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(x) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        print(x, ans)
        return ans
    except:
        print(x,'not found')
        return np.nan

df_names.loc[df_names['normalised_name'].isna(), 'normalised_name'] = df_names.loc[df_names['normalised_name'].isna(),'Name'].apply(CIRconvert)

CuSO4·5H2O not found
Ge25Sb10Bi5S60 not found
Y2O3 not found
Sapphire O=[Al]O[Al]=O
HfO2 not found
DMC CC(O)(c1ccc(Cl)cc1)c2ccc(Cl)cc2
MIBK CC(C)CC(C)=O
MgO [OH-]|[Mg++](|[OH-])(|[OH-])(|[OH-])(|[OH-])|[OH-]
NaFeSi2O6 not found
DAP Nc1cccc(N)n1
B40Bi60Sm1.0 not found
BiZnBo– not found
BFTZ not found
SiTi not found
polystyrene C=Cc1ccccc1
SnS2 not found
polycarbonate CC(C)(c1ccc(O)cc1)c2ccc(O)cc2
PVB CC(C)[C@H](CO)Nc1nc(Nc2ccc(C(O)=O)c(Cl)c2)c3ncn(C(C)C)c3n1
PVC ClC=C
Si–H(N2Si) not found
RbTi0.971Yb0.015Nb0.014OPO4 not found
TeEr1 not found
Si3N4 not found
BSMLN not found
As2S3 not found
SiO2–Nb2O5 not found
CdSnAs2 not found
Tb3Ga5O12 not found
BiFeO3 not found
SU-8 not found
TiO2-SiO2 not found
polypropylene CC=C
As30Se70 not found
Poly(N-vinylpyrrolidone) C=CN1CCCC1=O
Ca2Al2SiO7 not found
Mg3Al2Si3O12(P) not found
Sb2O3–GeO2 not found
B10P20T70 not found
polyurethane OCCO.OCCCCO.OC(=O)CCCCC(O)=O.O=C=Nc1ccc(Cc2ccc(cc2)N=C=O)cc1
oxyfluoride FOF
LaF3 [F-].[F-].[F-].[La+3]
Bi2O3.xSiO20.

Некоторые он тоже заменил неправильно, поправим в ручную

In [63]:
df_names.loc[df_names[df_names["Name"].isin(['MgO', 'CaO'])].index, 'normalised_name'] = np.nan

In [65]:
df_names.to_csv('df_names.csv', index=False)

Возвращаясь к первой проблеме, когда у нам просто нет смайлзов в различных библиотеках. Мы используем парсер и найдем смайлзы в странице поиска puchem, сохраним их и добавим сюда.

In [70]:
pubchem_smiles = pd.read_csv('/content/pubchem_smiles.csv').rename(columns={'0': 'smiles'})

In [71]:
pubchem_smiles

,Name,smiles
0,PbO-SiO2,NaN
1,PVAc,NaN
2,MoS2,S=[Mo]=S
3,CdTe,[Cd]=[Te]
4,N2 PDA,NaN
...,...,...
1032,CSO,C(=O)=S
1033,BPYb20,NaN
1034,YLiF4,[Li+].[F-].[F-].[F-].[F-].[Y+3]
1035,TiO2-AcAc-PEG,NaN


In [73]:
d = pd.merge(df_names, pubchem_smiles, how = 'left', on='Name')

In [77]:
df_names.loc[d[~d['smiles'].isna()].index, 'normalised_name'] = d[~d['smiles'].isna()]['smiles'].values

После всех правок у нас остались названия, которые либо дейтсвительно не присуствуют в выбранных библиотках или поисковиках, либо некооректно отображены. Лучше сначала поправить названия веществ. 

In [131]:
df_names[(df_names['check_smiles'].isna()) | (df_names['check_smiles'] is False)][300:]

,Name,normalised_name,check_smiles
1732,GaSe,[Ga]=[Se],NaN
1737,BBT2,NaN,NaN
1744,BAP-CTM-2,NaN,NaN
1755,Si0.236Ge0.725O0.039,NaN,NaN
1759,LiAlSi3O8,NaN,NaN
1764,LiLa(MoO4)2,NaN,NaN
1770,Ag–ZnO,NaN,NaN
1772,Ethyl-hexylalcohol,CCCCCC(O)CC,NaN
1773,ZnAlBiBTb1.0,NaN,NaN
1777,IPZO–,NaN,NaN


In [ ]:
# можно соединить смайлзы (проверить некоторые, точно ли можно их разбить на составляющие):
combine = {'Ge25Sb10Bi5S60': [],
'NaFeSi2O6': [],
'B40Bi60Sm1.0': [],
'BiZnBo-' : [],
'Si-H(N2Si)	' : [],
'RbTi0.971Yb0.015Nb0.014OPO4': [],
'TeEr1': [],
'SiO2-Nb2O5': [],
'Bi2Nd2Ti3O12': [],
'Li2O-SiO2': [],
'NaCl0.3KCl0.1KBr0.6' : [],
'Bi2O330SiO2xMF' : [],
'Mn1-xCuxFe2O4' : [], # x бери везде минимальное количество возможное
'TiO2-P2O5(H3PO4)' : [],
'Si0.75Ge0.25' : [],
'La0.7Sr0.3MnO3' : [],
'Ge14As20Se66': [], 
'As20S60Ge20': [], 
'WO3-xEr2O3': [],
'ZnSe(Te)': [],
'As2S3-GeS2-Sb2S3': [],
'B25Ti10':  [],
'CuGaS2': [],
'GaP0.9Sb0.1': [],
'GaSxSe1−x': [],
'Al0.593Ga0.25In0.031N': [],
'MgO-PbF2-SiO2': [],
'ZnGeP2': [],
'TiO2-SiO2': [],
'SiO2-TiO2	': [],
'BPbF-Sm-4': ['BPbF', 'Sm'], # 4 может быть ошибкой парсинга статьи
'Si-InP': [],
'YBG-S': [],
'TiO2-ZrO2': [],
'Al0.90Ga0.10As': [],
'C32H20N10Pb': [],
'PbO-TeO2': [],
'Na4Zr2Si3O12': [],
'Y6O5F8': [],
'Sb2O3-GeO2': [],
'CdSnAs2': [],
'BiFeO3': [],
'As30Se70': [],
'Ca2Al2SiO7': [],
'Mg3Al2Si3O12(P)': [],
'B10P20T70': [],
'Bi2O3.xSiO20.5Sm2O3': [],
'LiF-Sb2O3': [],
'As40Se60': [],
'GeSe5Tl4	': [],
'SrO-P2O5': [],
'PbO-B2O3': [],
'Si3-B	': [],
'Ge12.5Sb25Se62.5': [],
'AlAs': [],
'Cd0.60Se0.40': [],
'ZnSiAs2': ['Zn', 'Si', 'As', 'As'],
'S10T15HAlEr': [],
'Y2Te4O11': [],
'PbxGe42-xSe48Te10': [],
'LiFo': [],
'YAl3(BO3)4': [],
'Ge20As20S50Te10': [],
'BSiCaLa15': [], # число 15 может быть ошибкой парсинга а то неадкватно тут стоит
'HfTiO4': [],
'Cu1[As2(S0.5Se0.5)3]99': [], # число 99 может быть ошибкой парсинга
'La2Zr2O7': [], # может это оксид La и оксид Zr, не уверена что стоит разбивать на кислород La и Zr
'Bi2O3-B2O3': [],
'CsSnBr3': [],
'HS(CH2CH2O)nCH3': [],
'CdGe(P0.6As0.4)2': [],
'Y2Ti2O7': [],
'Fe3Al2Si3O12(A)': [], # конечная А может быть ошибкой парсинга
'SnTe': [],
'YK48': [], # число 48 может быть ошибкой парсинга или изотоп или реально 48?
'Ga15Se85': [],
'SiAlON': [],
'BBC2.5Er1.0': [],
'Se80Te18Pb2': [],
'BaO-TeO2': [],
'Ge2N2O': [],
'Pr BCR-2G LA-ICP-MS': [],
'K2O·2SiO2': ['K2O', 'SiO2', 'SiO2'],
'CaAl2B2O7': [],
'NiPcR8': [],
'BaGdGC': [],
'As-Se': [],
'PHI-6FZr30': [],
'Ni(D4EPC)2': [],
'PbF2-TeO2-B2O3-Sm2O3': [],
'N2Ac': [],
'Ge17As18S26Se39': [],
'H(ZrO2)': [],
'Cd50S50−xSex': [], # забей на х тут наверное
'Sn-Zno': [],
'BBW0': [],
'Rb2O-TiO2': [],
'AgGaTe2': [],
'ZrOSiCH': [],
'GaP0.3Sb0.7': [],
'Si1-H': [],
'NaSb3Br10': [],
'C3S32': [],
'HfO2 - TDHT': [],
'Cs2Hg3Cl8': [],
'BaO-B2O3-Al2O3-P2O5': [],
'AlFeO3': [],
'Se85−xTe15Bix': [],
'AgGaGeSe4': [],
'ZnSnP2': [],
'Na2O-90SiO2': [],
'AlOxNy_1': [],
'Y3Sc2Ga3O12YSGG': [],
'LiFe0.995Y0.0025Ag0.0025PO4': [],
'MoSe2': [],
'BaGdG': [],
'BPbF-3': [],
'Al-BSFs': [],
'K4Be3Si4O12': [],
'KTiOAsO4': [],
'Si0.024Ge0.734O0.242': [],
'HfMoN(L)': [],
'NbOx-06': [],
'Ge23.3As12Se64.7': [],
'GeO2-SiO2': [],
'NaCl0.2KCl0.2KBr0.6': [],
'Ho3Sb5O12': [],
'BSYCaEu2.0': [],
'AgInSnSe4': [],
'SiO2-HfO2': [],
'BSYCaEu2.5': [],
'Ca2SiO4': [],
'Au-CdSe': [],
'Al0.5In0.5FeO3': [],
'SrAl2B2O7': [],
'Ge14As28Se58': [],
'V2O5-P2O5': [],
'ZnS0.21Se0.79': [],
'GaTmO3': [],
'B1Pr': [],
'Cu3Ga5Se9': [],
'Ag-PVP': [],
'LiLuF': [],
'TlN': ['Tl', 'N'],
'Ge30As11Se49Te10': [],
'Gd1.7Eu0.1Li0.20O3': [],
'NaAlSi3O8': [],
'Ge29As12Se59': [],
'Ba3Tb(PO4)3': [],
'BBIR3': [],
'Dy2Sb2O7': [],
'Na2O-TeO2': [],
'LaSF9': [],
'BP1T-OMe': [],
'WO3-TeO2': [],
'PbCl2-PbBr2-TeO2': [],
'LiZnLaAPF': [],
'Sm2TeO6': [],
'As45S55': [],
'Na2[Al2Si6O10]2H2O': [],
'Zn1−xMnxO': [],
'Pb0.8Sn0.2Te': [],
'Sb2O3-P2O5': [],
'Al-Cu-O': ['Al', 'Cuo'],
'NdPb8': [],
'Sb2O3-Na2O': [],
'LiB3O5': [],
'Ge10Sb30S50Te10': [],
'As42S58': [],
'CABBCr3': [],
'Ni0.05Zn0.95O': [],
'Li2OPbO': [],
'GaP0.5Sb0.5': [],
'Zn0.50Mn0.50S': [],
'CdGeAs2': [],
'Gd1.9Eu0.1O3': [],
'Te20As40Se30I10': [],
'Y3Sc2Al3O12': [],
'SixOyNz': [],
'Na2O-Sb2O3-B2O3-SiO2': [],
'Y1−xErxAl3(BO3)4': [],
'HfMoN': [],
'BBFBNd0.25': [],
'Si0.236Ge0.725O0.039': [],
'LiAlSi3O8': [],
'LiLa(MoO4)2': [],
'Ag-ZnO': [],
'ZnAlBiBTb1.0': [],
'Ge25S65I10': [],
'Ge15As15Se17Te53': [],
'CdSe-PVP': [],
'C24H52O4Si': [],
'Al0.2Hf0.22O0.58': [],
'Ge21As12Se67': [],
'Se84-xTe15Bi1.0Pbx': [],
'SiON': [],
'CaSi2Al2O8(G)': [],
'Ge20Sb20S50Te10': [],
'GaSbS-PbI2': [],
'HgTe': [],
'ZnAlBiBTb3.0': [],
'CuGaGeSe4	': []
}

In [ ]:
check = {
    'BSMLN': '',
    'CN-325': '',
    'Y6O5F8': '',
    'BAP-T': '', 
    'BxC': '',
    'OCH2CH2': '',
    'ITO': '',
    'BFTZ': '',
    'CN-325': '',
    'C32H20N10Fe': '',
    'SU-8': '',
    'ZrON': '',
    'BMST': '',
    'CRSO': '',
    'P3HT': '',
    'NILS': '',
    'BBS-5': '',
    'Sn2S3': '',
    'C-Hn': '',
    'C7H5ClO': '',
    'CN': '',
    'BTPAm': '',
    'Lu2O3': '',
    'IZO': '',
    'CH2CH2CH2CH2CH2': '',
    'NPDR': '',
    'Fe2(SO4)3': '',
    'CH3(CH2)6COOH': '',
    'TeEr0.5': '',
    'BPRFQ': '',
    'Mg(PO3)2': '',
    'Gd2O3': '',
    'MoS2 –': '',
    'BAPT': '',
    'K-PG375': '',
    'BST115': '',
    'PEDOT': '',
    'C16H33(OCH2CH2)nOH': '',
    'PHSQ–VE1D': '',
    'Co-PAES': '',
    'KLTN': '',
    'C8H17C6H4CC[C6H2(C2H5)2CC]nC6H4C8H17': '',
    'CN978': '',
    'CdGeP2': '',
    'SbO3': '',
    'KZC': '',
    'PEI-4e': '',
    'NPCF': '',
    'CH3(CH2)6CH3': '',
    'NaFl': '',
    'BBS30': '',
    'SEBS': '',
    'BPC-C': '',
    'VB02': '',
    'P2VP': '',
    'BBT2': '',
    'BAP-CTM-2': '',
    'IPZO–': '',
    'BIPT': '',
    'BBGA4': '',
    'IPDI–': '',
    'BBTF1': '',
    'BPIE': '',
    'OV04': '',
    'KSO': ''
}

In [ ]:
#ошибка или добавление:
wrong = {
    'CuSO4·3H2O': 'O.O.O.[O-]S(=O)(=O)[O-].[Cu+2]',
    'nH': 'error',
    'aluminum–': '[Al]',
    'H(n)': 'not found'
    }

In [ ]:
errors = {
    'Al2O3n1' : 'Al2O3',
    'SiO2n': 'SiO2',
    'SiO2–': 'SiO2',
    'SiOx': 'SiO2',
    'GaP0.9Sb0.1' : 'GaSb',
    'Cu(II)' : 'Cu[2+]', 
    'SiN0.35': 'SiN', 
    'PrMn1.0': 'PrMn',
    'Na2SO4–': 'Na2SO4',
    'BS(1,1)': 'BS',
    'BS1': 'BS',
    'HxAc': 'Hexyl Acetate',
    'Ba–O2': 'BaO2',
    'BPbF-1': 'BPbF',
    'PrMn0.4': 'PrMn',
    'BiZnBo–': 'BiZnBo',
    'CN': 'CN', # check this
    'Cd–S': 'CdS',
    'N–H2': 'NH2',
    'NO2−': 'NO2',
    'N–H': 'NH',
    'Sr–F': 'SrF',
    'Li2O –': 'Li2O',
    'HfOx': 'HfO2',
    'HfSiOx': 'HfSiO',
    }

replace_by_nothing = [
    ' –', ' – polydimethylsiloxane', '-ormosil', 'nanorods', 'annealed', 
    'silicates', '0.35', '(4', 'nanoparticle', ' as-prepared', 'dipoles', 
    '−nn', '-3', '−x', '_2', '_aq_24', '-L', 'glassythin'
    ]

replace = {
    '–' : '-',
    '·' : '.',
}

Дальнейшая проверка датасета сильно зависит от задачи, которая перед нами стоит в рамках проектов. Самое базовое мы сделали:

* убрали дубликаты
* убрали строки с незаполненными названиями веществ
* насколько возмозможно в рамках поставленного времени привели в порядок смайлзы 

У нас еще остаются такие колонки как  `measurement_error`,`measurement_wavelength`,	`measurement_method`,	`raw_value`	и `specifier`. 
Похоже тут измерялся показатель преломения, определнными методами с указанной погрешностью по возможности. Так как не ясна задача работы с этим показателем, вне задач можно посмотреть разброс `raw_value` и посомтреть outliers и удалить их как невалидные данные.

## Descriptors

### RDkit

В RDkit можно вытащить десприпторы несколькими способами.
Первый это из `rdMolDescriptors`.

In [55]:
descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
get_descriptors = rdMolDescriptors.Properties(descriptor_names)
descriptors_dict = {'normalised_name': []}
descriptors_dict.update({name:[] for name in descriptor_names})
descriptors_df = pd.DataFrame(descriptors_dict)

def to_descriptors(smile):
    mol = Chem.MolFromSmiles(smile)
    descriptors = np.array([smile])
    
    if mol:
      descriptors = np.append(descriptors, np.array(get_descriptors.ComputeProperties(mol)))
    else:
      empty_arr = np.empty([len(descriptor_names),])
      empty_arr[:] = np.nan
      descriptors = np.append(descriptors, empty_arr)
    
    return pd.concat([descriptors_df, pd.DataFrame(dict(zip(['normalised_name'] + descriptor_names, descriptors)), index=[0])], ignore_index=True)

In [56]:
for name in df_names:
  descriptors_df = to_descriptors(name)

[18:46:07] SMILES Parse Error: syntax error while parsing: [['As',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['As',' for input: '[['As','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['O',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['O',' for input: '[['O','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['Al',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['Al',' for input: '[['Al','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['K',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['K',' for input: '[['K','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['C',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['C',' for input: '[['C','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['Er',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['Er',' for input: '[['Er','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['Li',
[18:46:07] SMILES Parse Error: Faile

In [57]:
descriptors_df

,normalised_name,exactmw,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumHeavyAtoms,NumAtoms,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
0,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,CCC=O,58.041864812,58.08,1.0,0.0,1.0,0.0,1.0,4.0,10.0,...,2.6927053408400363,1.3510573320459265,0.16666666666666669,0.16666666666666669,0.0,-0.33,3.67,2.67,1.6699999999999997,2.449725
2,"[['O', 1.0], ['PE', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,[AlH2],28.997188694,28.997999999999998,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.5324675324675323,1.5324675324675323,0.41051631295533775,4.0446626544187545,0.6291029211523357
4,O=[Al]O[Al]=O,101.94782112,101.961,3.0,0.0,2.0,0.0,3.0,5.0,5.0,...,2.379445409770841,0.9428090415820636,0.11111111111111116,0.11111111111111116,0.022680460581325737,0.4649350649350646,5.464935064935066,4.464935064935066,4.464935064935067,4.880116039804354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,C(C1C(C(C(C(O1)OCC2C(C(C(C(O2)O)O)O)O)O)O)O)O,342.11621152399994,342.297,11.0,8.0,4.0,8.0,11.0,23.0,45.0,...,11.99016988966061,7.073969678413754,3.9603220354868607,3.9603220354868607,2.4633653983706583,-0.43999999999999995,18.89235232228663,7.630403592249655,3.805051280688701,6.267664044609653
3468,C(C1C(C(C(C(O1)OC2(C(C(C(O2)CO)O)O)CO)O)O)O)O,342.11621152399994,342.297,11.0,8.0,5.0,8.0,11.0,23.0,45.0,...,12.042576132467905,7.049460184948061,4.070325339078977,4.070325339078977,2.6313159666862105,-0.43999999999999995,18.89235232228663,7.207281294562587,3.2268020699807716,5.92010858707402
3469,"[['O', 2.0], ['Si', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3470,CC1C(CC2CC(N=C3N2C1CN3)C(C4=CC(=O)NC(=O)N4)O)O...,415.11616901199994,415.4280000000001,12.0,5.0,4.0,5.0,9.0,28.0,49.0,...,14.901116352670037,8.894704407069693,5.673491239897029,5.673491239897029,4.417118841254056,-2.2199999999999998,19.111903235636287,6.686108592751132,3.4530865856864943,4.563723587414855


In [58]:
descriptors_df = pd.read_csv('/content/descriptors_df.csv')

Помимо этого, в RDkit есть модуль `MoleculeDescriptors`, который выдает около 200 дескрипторов 

In [59]:
des_list = [x[0] for x in Descriptors._descList]
descriptors_dict = {'normalised_name': []}
descriptors_dict.update({name:[] for name in des_list})
descriptors_df_2 = pd.DataFrame(descriptors_dict)

def to_descriptors(smile):
    mol = Chem.MolFromSmiles(smile)
    descriptors = np.array([smile])
    
    if mol:
      calculator = MoleculeDescriptors.MolecularDescriptorCalculator(des_list)
      descriptors = np.append(descriptors, list(calculator.CalcDescriptors(mol)))
    else:
      empty_arr = np.empty([len(des_list),])
      empty_arr[:] = np.nan
      descriptors = np.append(descriptors, empty_arr)
    
    return pd.concat([descriptors_df_2, pd.DataFrame(dict(zip(['normalised_name'] + des_list, descriptors)), index=[0])], ignore_index=True)

In [60]:
for name in df_names:
  descriptors_df_2 = to_descriptors(name)

[18:46:46] SMILES Parse Error: syntax error while parsing: [['As',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['As',' for input: '[['As','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['O',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['O',' for input: '[['O','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['Al',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['Al',' for input: '[['Al','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['K',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['K',' for input: '[['K','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['C',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['C',' for input: '[['C','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['Er',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['Er',' for input: '[['Er','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['Li',
[18:46:46] SMILES Parse Error: Faile

In [71]:
descriptors_df_2

,normalised_name,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,CCC=O,9.17361111111111,0.6388888888888888,9.17361111111111,0.6388888888888888,0.40325990291792796,58.08,52.032000000000004,58.041864812,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[['O', 1.0], ['PE', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,[AlH2],0.0,0.0,0.0,0.0,0.30172522635738636,28.997999999999998,26.982,28.997188694,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,O=[Al]O[Al]=O,9.211805555555555,-1.239583333333334,9.211805555555555,1.2395833333333335,0.4168020207505891,101.961,101.961,101.94782112,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,C(C1C(C(C(C(O1)OCC2C(C(C(C(O2)O)O)O)O)O)O)O)O,9.778871672125641,-1.7390736541530194,9.778871672125641,0.4675623582766437,0.24325846760776557,342.297,320.12100000000004,342.11621152399994,136.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3468,C(C1C(C(C(C(O1)OC2(C(C(C(O2)CO)O)O)CO)O)O)O)O,9.997825727513227,-2.221747921390779,9.997825727513227,0.6686328157806329,0.2378914345193498,342.297,320.12100000000004,342.11621152399994,136.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3469,"[['O', 2.0], ['Si', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3470,CC1C(CC2CC(N=C3N2C1CN3)C(C4=CC(=O)NC(=O)N4)O)O...,11.525320805059815,-4.591380511507224,11.525320805059815,0.05649111342067781,0.35121390661825996,415.4280000000001,394.2600000000001,415.11616901199994,154.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
descriptors_df = pd.merge(descriptors_df, descriptors_df_2, on='normalised_name')
descriptors_df = descriptors_df.drop_duplicates().reset_index(drop='True')
descriptors_df.to_csv('/content/descriptors_df.csv', index=False)

Так же можно использовать API PubChem, но он выдает только 39 десприпторов

In [104]:
properties = ['MolecularFormula', 'MolecularWeight', 'CanonicalSMILES', 'IsomericSMILES', 'InChI', 'InChIKey', 'IUPACName', 'XLogP', 'ExactMass', 
              'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 'HeavyAtomCount', 
              'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 'DefinedBondStereoCount', 
              'UndefinedBondStereoCount', 'CovalentUnitCount', 'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 'ZStericQuadrupole3D', 'FeatureCount3D', 
              'FeatureAcceptorCount3D', 'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 
              'ConformerModelRMSD3D', 'EffectiveRotorCount3D', 'ConformerCount3D']

descriptors_pub = pd.DataFrame()

for name in df_names:
  try:
    p = pcp.get_properties(properties, name, 'smiles', as_dataframe=True)
    p['normalised_name'] = name
    descriptors_pub = pd.concat([descriptors_pub, p], ignore_index=True)
  except Exception:
    print(name)
  

[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]
[['O', 1.0], ['PE', 1.0]]
[['Al', 2.0], ['O', 3.0]]
[['K', 2.0], ['O', 1.0]]
[['C', 2.0], ['H', 4.0], ['O', 1.0]]
[['Er', 2.0], ['O', 6.0], ['W', 1.0]]
[['Li', 2.0], ['O', 2.0], ['O-', 1.0], ['Si', 1.0]]
[['O', 2.0], ['Zr', 1.0]]
[['O', 2.0], ['Sn', 1.0]]
[['As', 2.0], ['Ge', 1.0], ['S', 8.0], ['Sb', 2.0]]
[['Cu', 1.0], ['Ga', 1.0], ['Se', 2.0]]
[['CL', 1.0], ['Ch', 1.0]]
[['Se', 1.0], ['Te', 1.0], ['Zn', 1.0]]
[['Cu', 1.0], ['I', 2.0]]
[['O', 2.0], ['Si', 1.0]]
[['He', 1.0], ['La', 1.0]]
[['Ba', 1.0], ['O', 4.0], ['W', 1.0]]
[['Cl', 1.0], ['Cs', 1.0]]
[['As', 20.0], ['Ge', 14.0], ['Se', 66.0]]
[['Ca', 1.0], ['Dy', 1.0]]
[['N', 0.35], ['Si', 1.0]]
[['H', 2.0], ['Odipoles', 1.0]]
[['Ni', 1.0], ['O', 1.0]]
[['F', 1.0], ['NM', 1.0]]
[['O', 3.0], ['Tb', 2.0]]
[['Ox', 1.0], ['Si', 1.0]]
[['As', 1.0], ['Ga', 1.0]]
[['Al', 1.0], ['Sb', 1.0]]
[['B', 25.0], ['Ti', 10.0]]
[['Se', 1.0], ['Zn', 1.0]]
[['S', 1.0], ['Zn', 1.0]]
[['In', 1.0], ['Sb', 1.0]]
[['

In [116]:
descriptors_df = pd.merge(descriptors_df, descriptors_pub, on='normalised_name', how='outer')
descriptors_df = descriptors_df.drop_duplicates().reset_index(drop=True)
descriptors_df.to_csv('/content/descriptors_df.csv', index=False)

In [13]:
descriptors_df = pd.read_csv('descriptors_df.csv')

Еще один способ получить много дескрипторов - через библиотеку `mordred`, она выдает около 1600 дескрипторов.

In [16]:
calc = Calculator(descriptors, ignore_3D=True)

def to_mol(smile):
  mol = Chem.MolFromSmiles(smile)
  if mol:
    return mol


mols = [to_mol(smi) for smi in df_names]
df_names = 
mols = [i for i in mols if i is not None]
descriptors_mor = calc.pandas(mols)
descriptors_mor

[20:29:42] SMILES Parse Error: syntax error while parsing: [['As',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['As',' for input: '[['As','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['O',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['O',' for input: '[['O','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['Al',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['Al',' for input: '[['Al','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['K',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['K',' for input: '[['K','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['C',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['C',' for input: '[['C','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['Er',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['Er',' for input: '[['Er','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['Li',
[20:29:42] SMILES Parse Error: Faile

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 12%|█▏        | 152/1281 [00:30<04:05,  4.60it/s][20:30:13] WARNING: not removing hydrogen atom without neighbors
[20:30:13] WARNING: not removing hydrogen atom without neighbors
[20:30:13] WARNING: not removing hydrogen atom without neighbors
 14%|█▎        | 174/1281 [00:34<03:02,  6.07it/s][20:30:17] WARNING: not removing hydrogen atom without neighbors
[20:30:17] WARNING: not removing hydrogen atom without neighbors
[20:30:17] WARNING: not removing hydrogen atom without neighbors
 19%|█▉        | 247/1281 [00:56<24:26,  1.42s/it]

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 21%|██        | 265/1281 [00:58<03:33,  4.75it/s][20:30:41] WARNING: not removing hydrogen atom without neighbors
[20:30:41] WARNING: not removing hydrogen atom without neighbors
[20:30:41] WARNING: not removing hydrogen atom without neighbors
 72%|███████▏  | 922/1281 [27:11<01:24,  4.25it/s][20:56:54] WARNING: not removing hydrogen atom without neighbors
[20:56:54] WARNING: not removing hydrogen atom without neighbors
[20:56:54] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 1281/1281 [28:43<00:00,  1.35s/it]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,2.121320,2.340100,0,0,4.472136,1.618034,3.236068,4.472136,1.118034,2.155909,...,5.509388,22.328143,58.041865,5.804186,10,1,10.0,8.0,2.5,1.250000
1,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.693147,...,0.000000,1.000000,28.997189,9.665730,0,0,0.0,0.0,divide by zero encountered in power (mZagreb1),0.000000
2,2.828427,3.146264,0,0,5.464102,1.732051,3.464102,5.464102,1.09282,2.390167,...,6.192362,25.583106,101.947821,20.389564,20,2,14.0,12.0,2.75,1.500000
3,4.242641,4.000000,0,0,8.0,2.0,4.0,8.0,1.333333,2.687624,...,7.627057,30.941317,138.961807,13.896181,27,3,24.0,24.0,1.5,1.500000
4,4.530370,5.004088,0,0,7.662988,2.052881,4.105762,7.662988,1.094713,2.766317,...,7.890957,32.688753,101.060255,6.316266,46,6,26.0,26.0,4.472222,1.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,6.651690,6.355891,0,0,10.79228,2.245827,4.491654,10.79228,1.199142,3.099448,...,8.806724,37.839725,233.954163,14.622135,84,10,42.0,46.0,4.083333,2.027778
1277,14.696938,14.696938,12,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,7.978311,57.944786,536.769646,18.509298,37000000054,0,72.0,54.0,divide by zero encountered in power (mZagreb1),6.000000
1278,3.863703,3.863703,0,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,6.311735,28.541146,325.866461,21.724431,1200000013,0,18.0,13.0,5.361111,2.000000
1279,8.623181,8.086503,0,0,13.878163,2.396777,4.70414,13.878163,1.261651,3.352605,...,9.317849,55.364239,151.049410,9.440588,138,14,58.0,68.0,3.694444,2.388889


In [1]:
# cols_to_use = descriptors_mor.columns.difference(descriptors_df.columns)
# df_5 = pd.merge(descriptors_df, descriptors_mor[cols_to_use], on='normalised_name', how='outer')
# df_5 = df_5.drop_duplicates().reset_index(drop=True)
# df_5
# # df_5.to_csv('/content/df_3.csv', index=False)

In [55]:
mols = [to_mol(smi) for smi in df_names]
df_names = df_names.reset_index(drop=True)
df_names = df_names[~pd.Series(mols).isna()].reset_index(drop=True)
descriptors_mor['normalised_name'] = df_names

In [56]:
descriptors_mor

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,normalised_name
0,2.121320,2.340100,0,0,4.472136,1.618034,3.236068,4.472136,1.118034,2.155909,...,22.328143,58.041865,5.804186,10,1,10.0,8.0,2.5,1.250000,CCC=O
1,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.693147,...,1.000000,28.997189,9.665730,0,0,0.0,0.0,divide by zero encountered in power (mZagreb1),0.000000,[AlH2]
2,2.828427,3.146264,0,0,5.464102,1.732051,3.464102,5.464102,1.09282,2.390167,...,25.583106,101.947821,20.389564,20,2,14.0,12.0,2.75,1.500000,O=[Al]O[Al]=O
3,4.242641,4.000000,0,0,8.0,2.0,4.0,8.0,1.333333,2.687624,...,30.941317,138.961807,13.896181,27,3,24.0,24.0,1.5,1.500000,N1PNP=NP1
4,4.530370,5.004088,0,0,7.662988,2.052881,4.105762,7.662988,1.094713,2.766317,...,32.688753,101.060255,6.316266,46,6,26.0,26.0,4.472222,1.777778,COC(=O)[C](C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,6.651690,6.355891,0,0,10.79228,2.245827,4.491654,10.79228,1.199142,3.099448,...,37.839725,233.954163,14.622135,84,10,42.0,46.0,4.083333,2.027778,CC1=CC(=C(C=C1)O)I
1277,14.696938,14.696938,12,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,57.944786,536.769646,18.509298,37000000054,0,72.0,54.0,divide by zero encountered in power (mZagreb1),6.000000,[Be+2].[Be+2].[Be+2].[O-][Si](=O)[O-].[O-][Si]...
1278,3.863703,3.863703,0,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,28.541146,325.866461,21.724431,1200000013,0,18.0,13.0,5.361111,2.000000,CC(C)=O.ICI
1279,8.623181,8.086503,0,0,13.878163,2.396777,4.70414,13.878163,1.261651,3.352605,...,55.364239,151.049410,9.440588,138,14,58.0,68.0,3.694444,2.388889,C1=NC2=C(N1)C(=O)NC(=N2)N


In [57]:
descriptors_mor.to_csv('descriptors_mor.csv', index=False)

In [1]:
!pip install drfp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 32.6 MB/s eta 0:00:00


In [3]:
from drfp import DrfpEncoder
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/dataset_smiles_descriptors+reaction_smiles.tsv', sep="\t")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6,10,11,12,24,31,34,37,38,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
def to_drfp(x):
    try:
        return DrfpEncoder.encode(x, n_folded_length=512)
    except OverflowError:
        print(x)

df['reaction_drfp'] = df['reaction_smiles'].apply(to_drfp)

In [6]:
df['reaction_drfp'] = DrfpEncoder.encode(df['reaction_smiles'].to_list(), n_folded_length=512)

In [7]:
df.shape

(108937, 45)

In [8]:
df['reaction_drfp']

0         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
2         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
3         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ...
4         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
108932    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
108933    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
108934    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
108935    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...
108936    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
Name: reaction_drfp, Length: 108937, dtype: object

In [9]:
df.to_csv('dataset_smiles_descriptors+reaction_fp.tsv', index=False, sep="\t")

In [10]:
df['reaction_drfp'][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,